In [ ]:
`using LaTeXStrings
sprotall = "sprot"
scop = "scop"
sprotscop = sprotall*'/'*scop
chr22 = "chromosome22"
proteins = chr22*"proteins"
pseudo = chr22*"pseudogenes"
rna = chr22*"rna"
shuf(x) = "\\textbf{shuffled}($(x))"
rev(x) = "\\textbf{reversed}($(x))"
reorder(x) = "\\textbf{reordered}($(x))"
mask(x) = "\\textbf{masked}($(x))"
tryptic(x) = "\\textbf{tryptic}($(x))"
permute(x) = "\\textbf{permuted}($(x))"
LPS(x) = "\\textbf{LPS}($(x))"
LCS(x, y) = "\\textbf{LCS}($(x),$(y))"
diff(x, y) = "$(x)-$(y)"
mean(x) = "\\textbf{mean}[$(x)]"

# Figure 1 - Sequence length distributions in SwissProt

### SCOP2 superfamily representative sequences
Download the SCOP2 superfamily representative sequences into the `data/` directory.

```
cd data
wget https://scop.mrc-lmb.cam.ac.uk/files/scop_sf_represeq_lib_latest.fa
```

For convenience, rename the file.

```
mv scop_sf_represeq_lib_latest.fa scop.fa
```

### Swiss-Prot
Download and unzip Swiss-Prot.

```
wget https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/complete/uniprot_sprot.fasta.gz
gzip -d uniprot_sprot.fasta.gz
mv uniprot_sprot.fasta data/sprot.fa
```

### Many-to-many alignment with BLAST
Construct a BLAST database from the SCOP2 superfamily representatives. First create a subdirectory `data/scopdb` to contain the files, then run `makeblastdb` to generate the database. 

```
mkdir scopdb
makeblastdb -dbtype prot -in scop.fa -out scopdb/scopsf -title scopsf
```

Align Swiss-Prot against the SCOP2 superfamily representative sequences. Set `num_threads` for your system.

```
cd ..
mkdir outputs
blastp -query data/sprot.fa -db data/scopdb/scop -outfmt "6 qseqid qstart qend sseqid sstart send" -num_threads 12 > outputs/sprot_scopsf.out
```

### Filter Swiss-Prot to sequences with >=95% identity to a representative sequence in the SCOP2 superfamily

In [ ]:
include("source/utils.jl")
include("source/io.jl")
include("source/pctid.jl")
include("source/alignment.jl")
using BioAlignments
using FASTX
using ProgressMeter
using Printf
# first, map Swiss-Prot sequence ids to their index in the record and likewise for the SCOP2 SF sequences
function lookupindex(names::AbstractVector)
    Dict(names[i] => i for i=1:length(names))
end
sprot = readfasta("data/sprot.fa")
sprotids = [split(descriptor, ' ')[1] for descriptor=description.(sprot)]
lookup_sprot_index = Dict(sprotids[i] => i for i=1:length(sprotids))
scop = readfasta("data/scop.fa")
scopids = [parse(Int, split(descriptor, ' ')[1]) for descriptor=description.(scop)]
lookup_scop_index = Dict(scopids[i] => i for i=1:length(scopids))
# each sequence in sprot will have a SCOP2 superfamily representative associated with it, as well as a pct id
n = length(sprot)
sprot_pct = zeros(AbstractFloat, n)
sprot_rep = zeros(Int, n)
# read the BLAST search results ...
columns = split("6 qseqid qstart qend sseqid sstart send", ' ')[2:end]
pathto_output = "outputs/sprot_scopsf.out"
outputs = readtable(pathto_output, columns)
qsubstrings = (FASTX.sequence(sprot[lookup_sprot_index[outputs.qseqid[i]]])[outputs.qstart[i]:outputs.qend[i]] for i=1:size(outputs)[1])
ssubstrings = (FASTX.sequence(scop[lookup_scop_index[outputs.sseqid[i]]])[outputs.sstart[i]:outputs.send[i]] for i=1:size(outputs)[1])

In [ ]:
# to get an accurate percent ID, we'll need to rerun alignments using BioAlignment.jl's global alignment.
pctid = align(Pairwise(), 
    qsubstrings, 
    ssubstrings, 
    model=GlobalAlignment(),
    verbose=true,
    formatter=x::PairwiseAlignmentResult -> percentid(x))

In [ ]:
for i=1:size(outputs)[1]
    pct = pctid[i]
    sprotidx = lookup_sprot_index[outputs.qseqid[i]]
    scopid = outputs.sseqid[i]
    if pct >= 0.95
        if pct > sprot_pct[sprotidx]
            sprot_pct[sprotidx] = pct
            sprot_rep[sprotidx] = scopid
        end
    end
end
represented = filter(i -> pctid[i] >= 0.95, sprot_pct)
representatives = filter(i -> pctid[i] == 0, sprot_rep)

In [ ]:
sum(sprot_pct .> 0.95) / length(sprot_pct)

### Plot

In [ ]:
# histogram
include("source/io.jl")

# Figure 2 - BLAST score distributions between Swiss-Prot variants

### Shuffling and reversing 

The command `esl-shuffle input.fa > output.fa` shuffles each sequence in a FASTA record. Using the `-r` flag, `esl-shuffle -r input.fa > output.fa` reverses each sequence. Generate the shuffled and reversed variants using EASEL.

```
esl-shuffle data/sprot.fa > data/shuf_sprot.fa
esl-shuffle data/sprot.fa -r > data/rev_sprot.fa

esl-shuffle data/sprot_scop.fa > data/shuf_sprot_scop.fa
esl-shuffle data/sprot_scop.fa -r > data/rev_sprot_scop.fa
```

Alternatively, generate the variants using Julia.

In [ ]:
# shuffling scripts
include("source/utils.jl")

### Classify Swiss-Prot sequences by their SCOP2 fold
Load the alignment results from Figure 1. Pair each sequence in `data/sprot_scop.fa` to its SCOP2 superfamily representative. Assign the Swiss-Prot sequence the fold of its SCOP2 superfamily representative and store these in a CSV.

In [ ]:
include("source/io.jl")

### Construct a derangement of `data/sprot_scop.fa` with respect to its SCOP2 fold classification.

In [ ]:
include("source/derangement.jl")

In [ ]:
# step-histogram
include("source/io.jl")

### Classify Swiss-Prot sequences by their SCOP2 fold

### Paired alignment with BLAST

# Figure 3 - BLAST scores by sequence length

### Masking with tantan

```
tantan -p data/sprot.fa > data/softmasked_sprot.fa 
```

### Masked paired alignment with BLAST

In [ ]:
include("source/utils.jl")
include("source/blast.jl")
using Main.BLAST

# foreach (query,target,mask) 
# generate singleton fastas
# BLAST.convert2blastmask
# BLAST.makeblastdb
# BLAST.blastp

### Plot

In [ ]:
# scatter plots
include("source/io.jl")

# Figure 4 - Reverse affinity of homologs

### BLOSUM90 variants

In [ ]:
include("source/io.jl")

### Inducing biorealistic mutations

In [ ]:
include("source/mutation.j")

### Paired alignment with BioAlignments.jl

In [ ]:
include("source/alignments.jl")

### Plot

In [ ]:
# box plots
include("source/io.jl")

# Figure 5 - LPS, LCS in Swiss-Prot

### Splice repetitive regions from Swiss-Prot using tantan masks
Rather than using lowercase letters for masks, configure tantan to mask with 'X' characters.

```
tantan -x X data/sprot.fa > data/masked_sprot.fa
```

### Generate LPS and LCS 
#### (using BioAlignments.jl for gapless exact alignment)

In [ ]:
include("source/palindrome.jl")
include("source/alignment.jl")

### Plot

In [ ]:
# scatter plots
include("source/io.jl")

# Figure 6 - LPS, LCS in DNA 

### Human Chromosome 22
Retrieve chromosome 22 on NCBI Nucleotide.
- <a href="https://www.ncbi.nlm.nih.gov/nuccore/NC_000022?report=fasta">GRCh38.p14 Primary Assembly /  NC_000022.11</a>

Download the sequence file in FASTA format using the `Send to` dropdown menu.
<img src="assets/ncbinucleotidesendto.png"/>

### NCBI Gene Annotations
Retrieve annotations for chromosome 22 e.g., using the NCBI Gene queries from <a href="https://en.wikipedia.org/wiki/Chromosome_22#cite_note-NCBI_coding-9">Wikipedia</a>.
- <a href="https://www.ncbi.nlm.nih.gov/gene?term=22%5BCHR%5D%20AND%20%22Homo%20sapiens%22%5BOrganism%5D%20AND%20%28%22genetype%20protein%20coding%22%5BProperties%5D%20AND%20alive%5Bprop%5D%29&cmd=DetailsSearch"> Protein-coding gene annotations </a>
- <a href="https://www.ncbi.nlm.nih.gov/gene?term=22%5BCHR%5D%20AND%20%22Homo%20sapiens%22%5BOrganism%5D%20AND%20%28%28%22genetype%20miscrna%22%5BProperties%5D%20OR%20%22genetype%20ncrna%22%5BProperties%5D%20OR%20%22genetype%20rrna%22%5BProperties%5D%20OR%20%22genetype%20trna%22%5BProperties%5D%20OR%20%22genetype%20scrna%22%5BProperties%5D%20OR%20%22genetype%20snrna%22%5BProperties%5D%20OR%20%22genetype%20snorna%22%5BProperties%5D%29%20NOT%20%22genetype%20protein%20coding%22%5BProperties%5D%20AND%20alive%5Bprop%5D%29&cmd=DetailsSearch"> Noncoding RNA annotations </a>
- <a href="https://www.ncbi.nlm.nih.gov/gene?term=22%5BCHR%5D%20AND%20%22Homo%20sapiens%22%5BOrganism%5D%20AND%20%28%22genetype%20pseudo%22%5BProperties%5D%20AND%20alive%5Bprop%5D%29&cmd=DetailsSearch"> Pseudogene annotations </a>
    
Download each annotation file in tabular text format using the `Send to` dropdown menu.
<img src="assets/ncbigenesendto.png"/>

### Masking with tantan
Run `tantan` on `data/ncbi_chr22_sequence.fasta` to mask repetitive and low-complexity regions; store the masked file in `data/ncbi_chr22_maskedsequence.fasta`. Rather than using lowercase letters for masks, configure tantan to mask with 'N' characters with the `-x N` argument. The proceeding script will automatically remove these along with the other 'N' characters already present in the chromosome.

```
tantan -x N data/ncbi_chr22_sequence.fasta > data/ncbi_chr22_maskedsequence.fasta
```

### Distribution of LPS in chromosome 22 [< 10 minutes]
Using the `lps` flag, generate the distribution of maximal palindromes in protein-coding, noncoding RNA, and pseudogene regions of chromosome 22.
```
julia -t 12 fig6.jl data/ncbi_chr22_sequence.fasta data/ncbi_chr22_protein.txt outputs/fig6_chr22palindromes_protein.dlm lps

julia -t 12 fig6.jl data/ncbi_chr22_sequence.fasta data/ncbi_chr22_pseudo.txt outputs/fig6_chr22palindromes_pseudo.dlm lps

julia -t 12 fig6.jl data/ncbi_chr22_sequence.fasta data/ncbi_chr22_rna.txt outputs/fig6_chr22palindromes_rna.dlm lps
```

Generate the LPS distribution with tantan masks. 

```
julia -t 12 fig6.jl data/ncbi_chr22_maskedsequence.fasta data/ncbi_chr22_protein.txt outputs/fig6_maskedchr22palindromes_protein.dlm lps

julia -t 12 fig6.jl data/ncbi_chr22_maskedsequence.fasta data/ncbi_chr22_pseudo.txt outputs/fig6_maskedchr22palindromes_pseudo.dlm lps

julia -t 12 fig6.jl data/ncbi_chr22_maskedsequence.fasta data/ncbi_chr22_rna.txt outputs/fig6_maskedchr22palindromes_rna.dlm lps
```

For a control, rerun each job with the `shuf` keyword to shuffle each region before the LPS is calculated.
```
julia -t 12 fig6.jl data/ncbi_chr22_sequence.fasta data/ncbi_chr22_protein.txt outputs/fig6_shuffledchr22palindromes_protein.dlm lps shuf

julia -t 12 fig6.jl data/ncbi_chr22_sequence.fasta data/ncbi_chr22_pseudo.txt outputs/fig6_shuffledchr22palindromes_pseudo.dlm lps shuf

julia -t 12 fig6.jl data/ncbi_chr22_sequence.fasta data/ncbi_chr22_rna.txt outputs/fig6_shuffledchr22palindromes_rna.dlm lps shuf
```

### Distribution of random LCS in chromosome 22 [ minutes]

In [ ]:
include("source/io.jl")
include("source/utils.jl")
include("source/alignment.jl")

### Plot

In [ ]:
using Plots
using DataFrames
using DelimitedFiles
using LaTeXStrings
include("source/utils.jl")
function plot_distribution_table!(distribution::Matrix{Int}, label, color, marker)
    seq_lengths, lps_lengths = eachcol(distribution)
    scatter!(seq_lengths, lps_lengths, label=label, color=color, markercolor=color, markerstrokecolor=color, markershape=marker, ms=2, ma=0.4)
end
dnalabels(x) = latexstring(mean(LPS(x)));

In [ ]:
chr22palindromes_protein = readdlm("outputs/fig6_chr22palindromes_protein.dlm", Int)
chr22palindromes_pseudo = readdlm("outputs/fig6_chr22palindromes_pseudo.dlm", Int)
chr22palindromes_rna = readdlm("outputs/fig6_chr22palindromes_rna.dlm", Int)
maskedchr22palindromes_protein = readdlm("outputs/fig6_maskedchr22palindromes_protein.dlm", Int)
maskedchr22palindromes_pseudo = readdlm("outputs/fig6_maskedchr22palindromes_pseudo.dlm", Int)
maskedchr22palindromes_rna = readdlm("outputs/fig6_maskedchr22palindromes_rna.dlm", Int);
shuffledchr22palindromes_protein = readdlm("outputs/fig6_shuffledchr22palindromes_protein.dlm", Int)
shuffledchr22palindromes_pseudo = readdlm("outputs/fig6_shuffledchr22palindromes_pseudo.dlm", Int)
shuffledchr22palindromes_rna = readdlm("outputs/fig6_shuffledchr22palindromes_rna.dlm", Int);

In [ ]:
seqlengths_and_palindromes = [
    shuffledchr22palindromes_protein, 
    shuffledchr22palindromes_pseudo, 
    shuffledchr22palindromes_rna,
    chr22palindromes_protein, 
    chr22palindromes_pseudo, 
    chr22palindromes_rna, 
    maskedchr22palindromes_protein, 
    maskedchr22palindromes_pseudo, 
    maskedchr22palindromes_rna,]
labels = dnalabels.([
    shuf(proteins),
    shuf(pseudo),
    shuf(rna),
    proteins,
    pseudo,
    rna,
    mask(proteins),
    mask(pseudo),
    mask(rna),])
colors = [:red, :red2, :red3, :blue, :blue2, :blue3, :green, :green2, :green3]
markers = [:+, :x, :star4, :dtriangle, :utriangle, :rtriangle, :star6, :star7, :star8];

In [ ]:
plot(#title="LPS in chromosome 22",
     ylabel="Average LPS length", 
     xlabel="Sequence length",
     xscale=:log,
     legend=:outertop,
     dpi=500)
subset = 1:length(seqlengths_and_palindromes)
plot_distribution_table!.(seqlengths_and_palindromes[subset], labels[subset], colors[subset], markers[subset])
savefig("figures/fig6_chr22palindromes.png")
plot!()

# Figure 7 - LPS, LCS in tryptic peptides of Swiss-Prot

### Generate LPS distributions of tryptic peptides [< 10 minutes]
Generate LPS distributions of tryptic peptides between lengths `5` and `50` in `data/sprot.fa`, `data/maskedsprot.fa`, and `data/shuf.fa` and write them to `data/sprot.fa outputs/fig7_trypticpalindromes_sprot.dlm`, `outputs/fig7_trypticpalindromes_maskedsprot.dlm`, and `outputs/fig7_trypticpalindromes_shuf.dlm`.
```
julia -t 12 fig7.jl data/sprot.fa outputs/fig7_trypticpalindromes_sprot.dlm 5 50

julia -t 12 fig7.jl data/masked_sprot.fa outputs/fig7_trypticpalindromes_maskedsprot.dlm 5 50

julia -t 12 fig7.jl data/shuf_sprot.fa outputs/fig7_trypticpalindromes_shuf.dlm 5 50
```

Also generate the LPS distribution for permuted tryptic peptides in `data\sprot.fa`. The keyword `permute` permutes each tryptic peptide before it is passed to the LPS algorithm; this allows us to compare palindromicity as a result of low-complexity tryptic peptides (e.g., dipeptide repeats are always palindromic) against palindromicity as a result of patterns that are not permutation invariant.
```
julia -t 12 fig7.jl data/sprot.fa outputs/fig7_permutedtrypticpalindromes_sprot.dlm 5 50 permute

julia -t 12 fig7.jl data/masked_sprot.fa outputs/fig7_permutedtrypticpalindromes_maskedsprot.dlm 5 50 permute

julia -t 12 fig7.jl data/shuf_sprot.fa outputs/fig7_permutedtrypticpalindromes_shuf.dlm 5 50 permute
```

### Plot

In [ ]:
using Plots
using DataFrames
using DelimitedFiles
using StatsBase
using LaTeXStrings
minlength = 5
maxlength = 50
function plot_distribution_matrix!(distribution::Matrix{Int}, label)
    global minlength, maxlength
    @assert maxlength == size(distribution)[1]
    distribution = maxlength .* distribution ./ sum(distribution)
    # calculate mean length palindrome for each sequence length
    avg_pal = [sum(collect(1:maxlength) .* distribution[seq, :]) for seq=minlength:maxlength]
    plot!(minlength:maxlength, avg_pal, label=label)
end
function plot_difference!(dist_A::Matrix{Int}, dist_B::Matrix{Int}, label)
    global minlength, maxlength
    @assert maxlength == size(dist_A)[1] == size(dist_B)[1]
    dist_A = maxlength .* dist_A ./ sum(dist_A)
    dist_B = maxlength .* dist_B ./ sum(dist_B)
    avg_pal_A =  [sum(collect(1:maxlength) .* dist_A[seq, :]) for seq=minlength:maxlength]
    avg_pal_B =  [sum(collect(1:maxlength) .* dist_B[seq, :]) for seq=minlength:maxlength]
    plot!(minlength:maxlength, avg_pal_A .- avg_pal_B, label=label)
end
trypticlpslabel(x) = latexstring(mean(LPS(tryptic(x))))
meandifflabel(x) = latexstring(mean(diff(LPS(tryptic(x)), LPS(permute(tryptic(x))))));

In [ ]:
sprot_dist = readdlm("outputs/fig7_trypticpalindromes_sprot.dlm", Int)
maskedsprot_dist = readdlm("outputs/fig7_trypticpalindromes_maskedsprot.dlm", Int)
shuf_dist = readdlm("outputs/fig7_trypticpalindromes_shuf.dlm", Int)
permutedsprot_dist = readdlm("outputs/fig7_permutedtrypticpalindromes_sprot.dlm", Int)
permutedmaskedsprot_dist = readdlm("outputs/fig7_permutedtrypticpalindromes_maskedsprot.dlm", Int)
permutedshuf_dist = readdlm("outputs/fig7_permutedtrypticpalindromes_shuf.dlm", Int);

In [ ]:
plot(#title="LPS in tryptic peptides between lengths 5 and 50",
     yrange=(1.5, 4.5),
     xlabel="Sequence length",
     xrange=(5, 50),
     xticks=([5, 10, 25, 50], [5, 10, 25, 50]),
     xscale=:log2,
     legend=:outertop,
     dpi=500)
plot_distribution_matrix!(sprot_dist, trypticlpslabel(sprotall))
plot_distribution_matrix!(shuf_dist, trypticlpslabel(shuf(sprotall)))
plot_distribution_matrix!(maskedsprot_dist, trypticlpslabel(mask(sprotall)))
savefig("figures/fig7_trypticpeptides.png")
plot!()

In [ ]:
plot(#title="Difference of mean length LPS between tryptic \npalindromes and permuted tryptic palindromes\n",
     #ylabel="Difference of average LPS length", 
     yrange=(-0.1,0.1),
     xlabel="Sequence length",
     #xscale=:log,
     #xrange=(5, 50),
     legend=:outertop,
     dpi=500)
plot_difference!(sprot_dist, permutedsprot_dist, meandifflabel(sprotall))
plot_difference!(shuf_dist, permutedshuf_dist, meandifflabel(shuf(sprotall)))
plot_difference!(maskedsprot_dist, permutedmaskedsprot_dist, meandifflabel(mask(sprotall)))
savefig("figures/fig7_reverseaffinity.png")
plot!()